# Paso 1. Analizar Dataframes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# 1. CARGA Y EXPLORACIÓN INICIAL DE DATOS
# =======================================

def load_dataset(file_path):
    """Carga el dataset con manejo de errores robusto"""
    try:
        # Intentar diferentes encodings
        encodings = ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']
        
        for encoding in encodings:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                print(f"✅ Dataset cargado exitosamente con encoding: {encoding}")
                return df
            except UnicodeDecodeError:
                continue
                
        # Si ningún encoding funciona, usar 'errors=replace'
        df = pd.read_csv(file_path, encoding='utf-8', errors='replace')
        print("⚠️  Dataset cargado con encoding UTF-8 y errores reemplazados")
        return df
        
    except FileNotFoundError:
        print(f"❌ Archivo no encontrado: {file_path}")
        return None
    except Exception as e:
        print(f"❌ Error al cargar el archivo: {str(e)}")
        return None

# Cargar el dataset
file_path = r"# === NOTE: Replace with local path ==="
df = load_dataset(file_path)

if df is not None:
    print("\n" + "="*60)
    print("INFORMACIÓN BÁSICA DEL DATASET")
    print("="*60)
    print(f"📊 Dimensiones: {df.shape[0]} filas x {df.shape[1]} columnas")
    print(f"📋 Columnas: {list(df.columns)}")
    print("\n📈 Primeras 5 filas:")
    print(df.head())

# 2. ANÁLISIS DE CALIDAD DE DATOS
# ===============================

def analyze_data_quality(df):
    """Análisis completo de calidad de datos"""
    
    print("\n" + "="*60)
    print("ANÁLISIS DE CALIDAD DE DATOS")
    print("="*60)
    
    # Información general
    print("\n🔍 INFORMACIÓN GENERAL:")
    print(df.info())
    
    # Estadísticas descriptivas
    print("\n📊 ESTADÍSTICAS DESCRIPTIVAS:")
    print(df.describe(include='all'))
    
    # Valores faltantes
    print("\n❌ VALORES FALTANTES:")
    missing_data = df.isnull().sum()
    missing_percent = (missing_data / len(df)) * 100
    missing_df = pd.DataFrame({
        'Columna': missing_data.index,
        'Valores_Faltantes': missing_data.values,
        'Porcentaje': missing_percent.values
    }).sort_values('Porcentaje', ascending=False)
    
    print(missing_df[missing_df['Valores_Faltantes'] > 0])
    
    # Duplicados
    duplicates = df.duplicated().sum()
    print(f"\n🔄 REGISTROS DUPLICADOS: {duplicates}")
    
    # Valores únicos por columna
    print("\n🎯 VALORES ÚNICOS POR COLUMNA:")
    for col in df.columns:
        unique_count = df[col].nunique()
        print(f"  {col}: {unique_count} valores únicos")
    
    return missing_df, duplicates

# Ejecutar análisis de calidad
if df is not None:
    missing_analysis, duplicates_count = analyze_data_quality(df)

# 3. ANÁLISIS ESPECÍFICO DE COLUMNAS CLAVE
# =========================================

def analyze_key_columns(df):
    """Análisis detallado de columnas importantes"""
    
    print("\n" + "="*60)
    print("ANÁLISIS DE COLUMNAS CLAVE")
    print("="*60)
    
    # Análisis de calificaciones numéricas
    if 'calificacion_numerica' in df.columns:
        print("\n📊 ANÁLISIS DE CALIFICACIONES:")
        calificaciones = df['calificacion_numerica'].dropna()
        
        print(f"  • Rango: {calificaciones.min():.2f} - {calificaciones.max():.2f}")
        print(f"  • Media: {calificaciones.mean():.2f}")
        print(f"  • Mediana: {calificaciones.median():.2f}")
        print(f"  • Desviación estándar: {calificaciones.std():.2f}")
        
        # Detectar outliers
        Q1 = calificaciones.quantile(0.25)
        Q3 = calificaciones.quantile(0.75)
        IQR = Q3 - Q1
        outliers = calificaciones[(calificaciones < Q1 - 1.5*IQR) | 
                                 (calificaciones > Q3 + 1.5*IQR)]
        print(f"  • Outliers detectados: {len(outliers)}")
        if len(outliers) > 0:
            print(f"    Valores: {sorted(outliers.tolist())}")
    
    # Análisis de nombres (normalizados vs originales)
    if 'nombre_original' in df.columns and 'nombre_normalizado' in df.columns:
        print("\n👤 ANÁLISIS DE NOMBRES:")
        nombres_orig = df['nombre_original'].dropna().nunique()
        nombres_norm = df['nombre_normalizado'].dropna().nunique()
        print(f"  • Nombres originales únicos: {nombres_orig}")
        print(f"  • Nombres normalizados únicos: {nombres_norm}")
        print(f"  • Reducción: {nombres_orig - nombres_norm} duplicados eliminados")
    
    # Análisis de departamentos
    if 'departamento' in df.columns:
        print("\n ANÁLISIS DE DEPARTAMENTOS:")
        dept_counts = df['departamento'].value_counts().head(10)
        print("  Top 10 departamentos:")
        for dept, count in dept_counts.items():
            print(f"    {dept}: {count} registros")
    
    # Análisis de comentarios
    if 'comentarios' in df.columns:
        print("\n ANÁLISIS DE COMENTARIOS:")
        comentarios_validos = df['comentarios'].dropna()
        print(f"  • Registros con comentarios: {len(comentarios_validos)}")
        
        # Extraer tags más comunes
        all_tags = []
        for comentario in comentarios_validos:
            if isinstance(comentario, str):
                tags = [tag.strip() for tag in comentario.split(',')]
                all_tags.extend(tags)
        
        tag_counts = Counter(all_tags)
        print("  • Top 10 tags más frecuentes:")
        for tag, count in tag_counts.most_common(10):
            print(f"    {tag}: {count} veces")

if df is not None:
    analyze_key_columns(df)

# 4. DETECCIÓN Y CORRECCIÓN DE ERRORES
# =====================================

def detect_and_fix_errors(df):
    """Detecta y corrige errores en el dataset"""
    
    print("\n" + "="*60)
    print("DETECCIÓN Y CORRECCIÓN DE ERRORES")
    print("="*60)
    
    df_corrected = df.copy()
    corrections_log = []
    
    # 1. Normalizar nombres de departamentos
    if 'departamento' in df_corrected.columns:
        print("\n🔧 Normalizando nombres de departamentos...")
        
        # Diccionario de normalizaciones
        dept_normalizations = {
            # Variaciones de DIVEC
            'divec': 'DIVEC',
            'Divec': 'DIVEC',
            'div. ingenierias': 'DIVEC',
            'Div. Ingenierías': 'DIVEC',
            
            # Variaciones de Matemáticas
            'matematicas': 'Matemáticas',
            'Matematicas': 'Matemáticas',
            'MATEMATICAS': 'Matemáticas',
            'Matem{aticas': 'Matemáticas',
            
            # Variaciones de Física
            'fisica': 'Física',
            'Fisica': 'Física',
            'FISICA': 'Física',
            'Fisicca': 'Física',
            
            # Variaciones de Química
            'quimica': 'Química',
            'Quimica': 'Química',
            'QUIMICA': 'Química',
            
            # Variaciones de Computación
            'computacion': 'Computación',
            'Computacion': 'Computación',
            'computacion e informatica': 'Computación e Informática',
            
            # Variaciones de Ingeniería
            'ingenieria': 'Ingeniería',
            'Ingenieria': 'Ingeniería',
            'ingenieria quimica': 'Ingeniería Química',
            'ingenieria industrial': 'Ingeniería Industrial',
            
            # CUCEI variaciones
            'cucei': 'CUCEI',
            'Cucei': 'CUCEI',
        }
        
        # Aplicar normalizaciones
        for old_name, new_name in dept_normalizations.items():
            mask = df_corrected['departamento'].str.contains(old_name, case=False, na=False)
            count = mask.sum()
            if count > 0:
                df_corrected.loc[mask, 'departamento'] = new_name
                corrections_log.append(f"Departamento: {old_name} → {new_name} ({count} registros)")
    
    # 2. Limpiar y validar calificaciones numéricas
    if 'calificacion_numerica' in df_corrected.columns:
        print("\n🔧 Validando calificaciones numéricas...")
        
        # Convertir a numérico, forzando errores a NaN
        df_corrected['calificacion_numerica'] = pd.to_numeric(
            df_corrected['calificacion_numerica'], errors='coerce'
        )
        
        # Validar rango (asumiendo escala 0-10)
        invalid_grades = df_corrected[
            (df_corrected['calificacion_numerica'] < 0) | 
            (df_corrected['calificacion_numerica'] > 10)
        ]['calificacion_numerica'].count()
        
        if invalid_grades > 0:
            # Clipear valores fuera del rango
            df_corrected['calificacion_numerica'] = df_corrected['calificacion_numerica'].clip(0, 10)
            corrections_log.append(f"Calificaciones: {invalid_grades} valores fuera de rango corregidos")
    
    # 3. Limpiar comentarios
    if 'comentarios' in df_corrected.columns:
        print("\n Limpiando comentarios...")
        
        def clean_comments(comment):
            if pd.isna(comment) or comment == '':
                return comment
            
            # Convertir a string y limpiar
            comment = str(comment)
            # Remover espacios extra
            comment = re.sub(r'\s+', ' ', comment).strip()
            # Estandarizar separadores
            comment = re.sub(r'[,;]+', ', ', comment)
            # Remover comas al final
            comment = comment.rstrip(', ')
            
            return comment
        
        original_comments = df_corrected['comentarios'].copy()
        df_corrected['comentarios'] = df_corrected['comentarios'].apply(clean_comments)
        
        # Contar cambios
        changes = (original_comments != df_corrected['comentarios']).sum()
        if changes > 0:
            corrections_log.append(f"Comentarios: {changes} registros limpiados")
    
    # 4. Eliminar duplicados exactos
    print("\n Eliminando duplicados...")
    original_count = len(df_corrected)
    df_corrected = df_corrected.drop_duplicates()
    duplicates_removed = original_count - len(df_corrected)
    
    if duplicates_removed > 0:
        corrections_log.append(f"Duplicados: {duplicates_removed} registros eliminados")
    
    # Mostrar resumen de correcciones
    print("\n RESUMEN DE CORRECCIONES:")
    if corrections_log:
        for correction in corrections_log:
            print(f"  ✅ {correction}")
    else:
        print("    No se requirieron correcciones")
    
    return df_corrected, corrections_log

if df is not None:
    df_corrected, corrections = detect_and_fix_errors(df)

# 5. ANÁLISIS ESTADÍSTICO AVANZADO
# =================================

def advanced_statistical_analysis(df):
    """Análisis estadístico avanzado del dataset"""
    
    print("\n" + "="*60)
    print("ANÁLISIS ESTADÍSTICO AVANZADO")
    print("="*60)
    
    # Crear figura con múltiples subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle('Análisis Estadístico - Dataset de Evaluaciones de Profesores', fontsize=16)
    
    # 1. Distribución de calificaciones
    if 'calificacion_numerica' in df.columns:
        calificaciones = df['calificacion_numerica'].dropna()
        
        # Histograma
        axes[0,0].hist(calificaciones, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        axes[0,0].set_title('Distribución de Calificaciones')
        axes[0,0].set_xlabel('Calificación')
        axes[0,0].set_ylabel('Frecuencia')
        axes[0,0].axvline(calificaciones.mean(), color='red', linestyle='--', 
                         label=f'Media: {calificaciones.mean():.2f}')
        axes[0,0].legend()
        
        # Box plot
        axes[0,1].boxplot(calificaciones)
        axes[0,1].set_title('Box Plot - Calificaciones')
        axes[0,1].set_ylabel('Calificación')
    
    # 2. Top departamentos
    if 'departamento' in df.columns:
        dept_counts = df['departamento'].value_counts().head(10)
        axes[0,2].barh(range(len(dept_counts)), dept_counts.values)
        axes[0,2].set_yticks(range(len(dept_counts)))
        axes[0,2].set_yticklabels(dept_counts.index, fontsize=8)
        axes[0,2].set_title('Top 10 Departamentos')
        axes[0,2].set_xlabel('Número de Profesores')
    
    # 3. Análisis de comentarios por calificación
    if 'comentarios' in df.columns and 'calificacion_numerica' in df.columns:
        # Crear categorías de calificación
        df_temp = df.copy()
        df_temp = df_temp.dropna(subset=['calificacion_numerica'])
        df_temp['categoria_calif'] = pd.cut(df_temp['calificacion_numerica'], 
                                           bins=[0, 5, 7, 8.5, 10], 
                                           labels=['Baja (0-5)', 'Media (5-7)', 'Alta (7-8.5)', 'Excelente (8.5-10)'])
        
        categoria_counts = df_temp['categoria_calif'].value_counts()
        axes[1,0].pie(categoria_counts.values, labels=categoria_counts.index, autopct='%1.1f%%')
        axes[1,0].set_title('Distribución por Categoría de Calificación')
    
    # 4. Análisis temporal (si hay fechas) o análisis de completitud
    completitud = df.count() / len(df) * 100
    axes[1,1].bar(range(len(completitud)), completitud.values)
    axes[1,1].set_xticks(range(len(completitud)))
    axes[1,1].set_xticklabels(completitud.index, rotation=45, ha='right', fontsize=8)
    axes[1,1].set_title('Completitud de Datos por Columna (%)')
    axes[1,1].set_ylabel('Porcentaje de Completitud')
    
    # 5. Correlación entre variables numéricas
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 1:
        corr_matrix = df[numeric_cols].corr()
        im = axes[1,2].imshow(corr_matrix, cmap='coolwarm', aspect='auto')
        axes[1,2].set_xticks(range(len(corr_matrix.columns)))
        axes[1,2].set_yticks(range(len(corr_matrix.columns)))
        axes[1,2].set_xticklabels(corr_matrix.columns, rotation=45, ha='right', fontsize=8)
        axes[1,2].set_yticklabels(corr_matrix.columns, fontsize=8)
        axes[1,2].set_title('Matriz de Correlación')
        plt.colorbar(im, ax=axes[1,2])
    
    plt.tight_layout()
    plt.show()
    
    return fig

if df is not None:
    fig = advanced_statistical_analysis(df_corrected)

# 6. ANÁLISIS DE SENTIMIENTOS EN COMENTARIOS
# ===========================================

def analyze_sentiment_comments(df):
    """Análisis de sentimientos en comentarios"""
    
    print("\n" + "="*60)
    print("ANÁLISIS DE SENTIMIENTOS EN COMENTARIOS")
    print("="*60)
    
    if 'comentarios' not in df.columns:
        print("No hay columna de comentarios para analizar")
        return
    
    # Definir diccionarios de sentimientos
    positive_keywords = [
        'EXCELENTE', 'BUENA', 'BRINDA APOYO', 'INSPIRACIONAL', 
        'RESPETADO', 'CÓMICO', 'TOMARÍA SU CLASE OTRA VEZ'
    ]
    
    negative_keywords = [
        'CALIFICA DURO', 'DIFÍCIL', 'LARGAS', 'MUCHAS TAREAS',
        'EXÁMENES SORPRESA', 'OBLIGATORIA'
    ]
    
    neutral_keywords = [
        'BARCO', 'POCOS EXÁMENES', 'PARTICIPACIÓN IMPORTA', 
        'ASPECTOS DE CALIFICACIÓN CLAROS'
    ]
    
    # Analizar sentimientos
    sentiment_scores = []
    comentarios_validos = df['comentarios'].dropna()
    
    for comentario in comentarios_validos:
        if isinstance(comentario, str):
            positive_count = sum(1 for keyword in positive_keywords if keyword in comentario.upper())
            negative_count = sum(1 for keyword in negative_keywords if keyword in comentario.upper())
            neutral_count = sum(1 for keyword in neutral_keywords if keyword in comentario.upper())
            
            # Calcular score de sentimiento
            total_keywords = positive_count + negative_count + neutral_count
            if total_keywords > 0:
                sentiment_score = (positive_count - negative_count) / total_keywords
            else:
                sentiment_score = 0
            
            sentiment_scores.append({
                'sentimiento_score': sentiment_score,
                'positivo': positive_count,
                'negativo': negative_count,
                'neutral': neutral_count
            })
        else:
            sentiment_scores.append({
                'sentimiento_score': 0,
                'positivo': 0,
                'negativo': 0,
                'neutral': 0
            })
    
    # Crear DataFrame de sentimientos
    sentiment_df = pd.DataFrame(sentiment_scores)
    
    # Estadísticas
    print(f"\n ESTADÍSTICAS DE SENTIMIENTO:")
    print(f"  • Score promedio: {sentiment_df['sentimiento_score'].mean():.3f}")
    print(f"  • Comentarios positivos: {(sentiment_df['sentimiento_score'] > 0.1).sum()}")
    print(f"  • Comentarios negativos: {(sentiment_df['sentimiento_score'] < -0.1).sum()}")
    print(f"  • Comentarios neutrales: {(abs(sentiment_df['sentimiento_score']) <= 0.1).sum()}")
    
    # Visualización
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.hist(sentiment_df['sentimiento_score'], bins=30, alpha=0.7, color='lightgreen')
    plt.title('Distribución de Scores de Sentimiento')
    plt.xlabel('Score de Sentimiento')
    plt.ylabel('Frecuencia')
    plt.axvline(0, color='red', linestyle='--', label='Neutral')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    sentiment_categories = ['Positivo', 'Neutral', 'Negativo']
    sentiment_counts = [
        (sentiment_df['sentimiento_score'] > 0.1).sum(),
        (abs(sentiment_df['sentimiento_score']) <= 0.1).sum(),
        (sentiment_df['sentimiento_score'] < -0.1).sum()
    ]
    plt.pie(sentiment_counts, labels=sentiment_categories, autopct='%1.1f%%', 
            colors=['lightgreen', 'lightgray', 'lightcoral'])
    plt.title('Distribución de Sentimientos')
    
    plt.tight_layout()
    plt.show()
    
    return sentiment_df

if df is not None:
    sentiment_analysis = analyze_sentiment_comments(df_corrected)

# 7. ANÁLISIS AVANZADO DE PATRONES
# =================================

def advanced_pattern_analysis(df):
    """Análisis avanzado de patrones en el dataset"""
    
    print("\n" + "="*60)
    print("ANÁLISIS AVANZADO DE PATRONES")
    print("="*60)
    
    # 1. Análisis de profesores con múltiples registros
    if 'nombre_normalizado' in df.columns:
        profesor_counts = df['nombre_normalizado'].value_counts()
        profesores_multiples = profesor_counts[profesor_counts > 1]
        
        print(f"\n👥 PROFESORES CON MÚLTIPLES REGISTROS:")
        print(f"  • Total profesores únicos: {len(profesor_counts)}")
        print(f"  • Profesores con múltiples registros: {len(profesores_multiples)}")
        print(f"  • Top 5 profesores con más registros:")
        
        for profesor, count in profesores_multiples.head().items():
            print(f"    - {profesor}: {count} registros")
    
    # 2. Análisis de correlación entre calificación y sentimientos
    if 'calificacion_numerica' in df.columns and 'comentarios' in df.columns:
        print(f"\n CORRELACIÓN CALIFICACIÓN-COMENTARIOS:")
        
        # Crear dataset temporal con análisis de sentimientos
        df_temp = df.dropna(subset=['calificacion_numerica', 'comentarios']).copy()
        
        # Calcular métricas de comentarios
        df_temp['num_comentarios'] = df_temp['comentarios'].str.count(',') + 1
        df_temp['comentarios_positivos'] = df_temp['comentarios'].str.count('EXCELENTE|BUENA|APOYO|INSPIRACIONAL')
        df_temp['comentarios_negativos'] = df_temp['comentarios'].str.count('DURO|DIFÍCIL|LARGAS|MUCHAS TAREAS')
        
        # Correlaciones
        correlations = {
            'Calificación vs Num Comentarios': df_temp['calificacion_numerica'].corr(df_temp['num_comentarios']),
            'Calificación vs Comentarios Positivos': df_temp['calificacion_numerica'].corr(df_temp['comentarios_positivos']),
            'Calificación vs Comentarios Negativos': df_temp['calificacion_numerica'].corr(df_temp['comentarios_negativos'])
        }
        
        for desc, corr in correlations.items():
            print(f"    {desc}: {corr:.3f}")
    
    # 3. Análisis de distribución por departamento
    if 'departamento' in df.columns and 'calificacion_numerica' in df.columns:
        print(f"\n ANÁLISIS POR DEPARTAMENTO:")
        
        dept_stats = df.groupby('departamento')['calificacion_numerica'].agg(['count', 'mean', 'std']).round(2)
        dept_stats = dept_stats.sort_values('mean', ascending=False).head(10)
        
        print("  Top 10 departamentos por calificación promedio:")
        for dept, row in dept_stats.iterrows():
            if pd.notna(row['mean']):
                print(f"    {dept}: {row['mean']:.2f} ± {row['std']:.2f} ({int(row['count'])} prof.)")

def detect_data_anomalies(df):
    """Detecta anomalías específicas en el dataset educativo"""
    
    print("\n" + "="*60)
    print("DETECCIÓN DE ANOMALÍAS ESPECÍFICAS")
    print("="*60)
    
    anomalies = []
    
    # 1. Profesores con calificaciones extremas vs comentarios
    if 'calificacion_numerica' in df.columns and 'comentarios' in df.columns:
        # Profesores con calificación alta pero sin comentarios positivos
        df_temp = df.dropna(subset=['calificacion_numerica', 'comentarios'])
        high_grade_no_positive = df_temp[
            (df_temp['calificacion_numerica'] >= 8.5) & 
            (~df_temp['comentarios'].str.contains('EXCELENTE|BUENA|APOYO|INSPIRACIONAL', na=False))
        ]
        
        if len(high_grade_no_positive) > 0:
            anomalies.append(f" {len(high_grade_no_positive)} profesores con calificación alta (≥8.5) pero sin comentarios claramente positivos")
        
        # Profesores con calificación baja pero comentarios positivos
        low_grade_positive = df_temp[
            (df_temp['calificacion_numerica'] <= 5.0) & 
            (df_temp['comentarios'].str.contains('EXCELENTE|BUENA|APOYO|INSPIRACIONAL', na=False))
        ]
        
        if len(low_grade_positive) > 0:
            anomalies.append(f"  {len(low_grade_positive)} profesores con calificación baja (≤5.0) pero con comentarios positivos")
    
    # 2. Registros con información incompleta crítica
    critical_missing = df[
        df[['nombre_normalizado', 'departamento', 'calificacion_numerica']].isnull().any(axis=1)
    ]
    
    if len(critical_missing) > 0:
        anomalies.append(f" {len(critical_missing)} registros con información crítica faltante (nombre, departamento o calificación)")
    
    # 3. Nombres con caracteres especiales o formato inusual
    if 'nombre_original' in df.columns:
        unusual_names = df[df['nombre_original'].str.contains(r'[^\w\s,.-]|^\d', na=False, regex=True)]
        if len(unusual_names) > 0:
            anomalies.append(f" {len(unusual_names)} nombres con formato inusual o caracteres especiales")
    
    # Mostrar anomalías encontradas
    if anomalies:
        print("\n ANOMALÍAS DETECTADAS:")
        for anomaly in anomalies:
            print(f"  {anomaly}")
    else:
        print("\n No se detectaron anomalías significativas")
    
    return anomalies

def create_data_quality_dashboard(df_original, df_corrected):
    """Crea un dashboard visual de calidad de datos"""
    
    print("\n" + "="*60)
    print("DASHBOARD DE CALIDAD DE DATOS")
    print("="*60)
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle('Dashboard de Calidad - Antes y Después de Correcciones', fontsize=16, fontweight='bold')
    
    # 1. Comparación de valores faltantes
    missing_orig = df_original.isnull().sum().sort_values(ascending=False)
    missing_corr = df_corrected.isnull().sum().sort_values(ascending=False)
    
    x = range(len(missing_orig))
    axes[0,0].bar([i-0.2 for i in x], missing_orig.values, width=0.4, label='Original', alpha=0.7)
    axes[0,0].bar([i+0.2 for i in x], missing_corr.values, width=0.4, label='Corregido', alpha=0.7)
    axes[0,0].set_title('Valores Faltantes')
    axes[0,0].set_xticks(x)
    axes[0,0].set_xticklabels(missing_orig.index, rotation=45, ha='right', fontsize=8)
    axes[0,0].legend()
    axes[0,0].set_ylabel('Cantidad')
    
    # 2. Comparación de registros totales
    sizes_orig = [df_original.shape[0] - df_corrected.shape[0], df_corrected.shape[0]]
    labels_orig = ['Eliminados', 'Conservados']
    axes[0,1].pie(sizes_orig, labels=labels_orig, autopct='%1.1f%%', startangle=90)
    axes[0,1].set_title('Registros: Original vs Corregido')
    
    # 3. Distribución de calificaciones (comparación)
    if 'calificacion_numerica' in df_original.columns:
        calif_orig = df_original['calificacion_numerica'].dropna()
        calif_corr = df_corrected['calificacion_numerica'].dropna()
        
        axes[0,2].hist(calif_orig, bins=20, alpha=0.5, label='Original', density=True)
        axes[0,2].hist(calif_corr, bins=20, alpha=0.5, label='Corregido', density=True)
        axes[0,2].set_title('Distribución de Calificaciones')
        axes[0,2].set_xlabel('Calificación')
        axes[0,2].set_ylabel('Densidad')
        axes[0,2].legend()
    
    # 4. Completitud por columna
    completitud_orig = (df_original.count() / len(df_original) * 100)
    completitud_corr = (df_corrected.count() / len(df_corrected) * 100)
    
    x = range(len(completitud_orig))
    axes[0,3].plot(x, completitud_orig.values, 'o-', label='Original', markersize=6)
    axes[0,3].plot(x, completitud_corr.values, 's-', label='Corregido', markersize=6)
    axes[0,3].set_title('Completitud por Columna (%)')
    axes[0,3].set_xticks(x)
    axes[0,3].set_xticklabels(completitud_orig.index, rotation=45, ha='right', fontsize=8)
    axes[0,3].set_ylabel('Porcentaje')
    axes[0,3].legend()
    axes[0,3].grid(True, alpha=0.3)
    
    # 5. Top departamentos (corregido)
    if 'departamento' in df_corrected.columns:
        dept_counts = df_corrected['departamento'].value_counts().head(8)
        axes[1,0].barh(range(len(dept_counts)), dept_counts.values)
        axes[1,0].set_yticks(range(len(dept_counts)))
        axes[1,0].set_yticklabels(dept_counts.index, fontsize=9)
        axes[1,0].set_title('Top Departamentos (Datos Corregidos)')
        axes[1,0].set_xlabel('Número de Registros')
    
    # 6. Distribución de longitud de comentarios
    if 'comentarios' in df_corrected.columns:
        comment_lengths = df_corrected['comentarios'].dropna().str.len()
        axes[1,1].hist(comment_lengths, bins=30, alpha=0.7, color='lightgreen')
        axes[1,1].set_title('Distribución de Longitud de Comentarios')
        axes[1,1].set_xlabel('Caracteres')
        axes[1,1].set_ylabel('Frecuencia')
        axes[1,1].axvline(comment_lengths.mean(), color='red', linestyle='--', 
                         label=f'Media: {comment_lengths.mean():.0f}')
        axes[1,1].legend()
    
    # 7. Calificaciones por departamento (top 6)
    if 'departamento' in df_corrected.columns and 'calificacion_numerica' in df_corrected.columns:
        top_depts = df_corrected['departamento'].value_counts().head(6).index
        dept_data = [df_corrected[df_corrected['departamento'] == dept]['calificacion_numerica'].dropna() 
                    for dept in top_depts]
        
        axes[1,2].boxplot(dept_data, labels=[d[:15] + '...' if len(d) > 15 else d for d in top_depts])
        axes[1,2].set_title('Calificaciones por Departamento (Top 6)')
        axes[1,2].set_ylabel('Calificación')
        axes[1,2].tick_params(axis='x', rotation=45, labelsize=8)
    
    # 8. Métricas de calidad general
    quality_metrics = {
        'Completitud\nPromedio': (df_corrected.count().sum() / (df_corrected.shape[0] * df_corrected.shape[1])) * 100,
        'Duplicados\n(%)': (df_corrected.duplicated().sum() / len(df_corrected)) * 100,
        'Registros\nVálidos (%)': (len(df_corrected) / len(df_original)) * 100,
        'Calificación\nPromedio': df_corrected['calificacion_numerica'].mean() if 'calificacion_numerica' in df_corrected.columns else 0
    }
    
    axes[1,3].bar(quality_metrics.keys(), quality_metrics.values(), 
                  color=['skyblue', 'lightcoral', 'lightgreen', 'gold'])
    axes[1,3].set_title('Métricas de Calidad')
    axes[1,3].set_ylabel('Porcentaje/Valor')
    axes[1,3].tick_params(axis='x', rotation=45, labelsize=9)
    
    # Añadir valores en las barras
    for i, (key, value) in enumerate(quality_metrics.items()):
        axes[1,3].text(i, value + max(quality_metrics.values()) * 0.01, 
                      f'{value:.1f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return fig

# 8. REPORTE FINAL Y EXPORTACIÓN
# ===============================

def generate_final_report(df_original, df_corrected, corrections, sentiment_df=None):
    """Genera reporte final con todas las métricas y estadísticas"""
    
    print("\n" + "="*80)
    print("REPORTE FINAL - ANÁLISIS Y CORRECCIÓN DE DATASET")
    print("="*80)
    
    print(f"\n📊 RESUMEN EJECUTIVO:")
    print(f"  • Dataset original: {df_original.shape[0]} filas × {df_original.shape[1]} columnas")
    print(f"  • Dataset corregido: {df_corrected.shape[0]} filas × {df_corrected.shape[1]} columnas")
    print(f"  • Registros eliminados: {df_original.shape[0] - df_corrected.shape[0]}")
    
    # Calidad de datos
    print(f"\n🎯 CALIDAD DE DATOS:")
    completitud_original = (df_original.count().sum() / (df_original.shape[0] * df_original.shape[1])) * 100
    completitud_corregida = (df_corrected.count().sum() / (df_corrected.shape[0] * df_corrected.shape[1])) * 100
    print(f"  • Completitud original: {completitud_original:.1f}%")
    print(f"  • Completitud corregida: {completitud_corregida:.1f}%")
    
    # Estadísticas de calificaciones
    if 'calificacion_numerica' in df_corrected.columns:
        calificaciones = df_corrected['calificacion_numerica'].dropna()
        print(f"\n⭐ ESTADÍSTICAS DE CALIFICACIONES:")
        print(f"  • Total de evaluaciones: {len(calificaciones)}")
        print(f"  • Calificación promedio: {calificaciones.mean():.2f}")
        print(f"  • Rango: {calificaciones.min():.2f} - {calificaciones.max():.2f}")
        print(f"  • Profesores con calificación ≥ 8.0: {(calificaciones >= 8.0).sum()} ({(calificaciones >= 8.0).mean()*100:.1f}%)")
    
    # Top departamentos
    if 'departamento' in df_corrected.columns:
        print(f"\n🏢 TOP 5 DEPARTAMENTOS:")
        top_depts = df_corrected['departamento'].value_counts().head()
        for i, (dept, count) in enumerate(top_depts.items(), 1):
            print(f"  {i}. {dept}: {count} profesores")
    
    # Correcciones aplicadas
    print(f"\n🔧 CORRECCIONES APLICADAS:")
    if corrections:
        for correction in corrections:
            print(f"  ✅ {correction}")
    else:
        print("    No se requirieron correcciones")
    
    # Análisis de sentimientos
    if sentiment_df is not None:
        print(f"\n💭 ANÁLISIS DE SENTIMIENTOS:")
        positive_pct = (sentiment_df['sentimiento_score'] > 0.1).mean() * 100
        negative_pct = (sentiment_df['sentimiento_score'] < -0.1).mean() * 100
        neutral_pct = (abs(sentiment_df['sentimiento_score']) <= 0.1).mean() * 100
        print(f"  • Comentarios positivos: {positive_pct:.1f}%")
        print(f"  • Comentarios negativos: {negative_pct:.1f}%")
        print(f"  • Comentarios neutrales: {neutral_pct:.1f}%")
    
    print(f"\n📋 RECOMENDACIONES:")
    print("  1. Implementar validación en tiempo real para futuras entradas")
    print("  2. Establecer estándares para nombres de departamentos")
    print("  3. Mejorar la recolección de datos para reducir valores faltantes")
    print("  4. Considerar implementar un sistema de feedback estructurado")
    
    return {
        'original_shape': df_original.shape,
        'corrected_shape': df_corrected.shape,
        'completitud_original': completitud_original,
        'completitud_corregida': completitud_corregida,
        'corrections': corrections
    }

def export_cleaned_dataset(df, output_path):
    """Exporta el dataset limpio"""
    try:
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"\n💾 Dataset limpio exportado a: {output_path}")
        return True
    except Exception as e:
        print(f"\n❌ Error al exportar: {str(e)}")
        return False

# Ejecutar análisis avanzado
if df is not None:
    advanced_pattern_analysis(df_corrected)
    anomalies = detect_data_anomalies(df_corrected)
    dashboard_fig = create_data_quality_dashboard(df, df_corrected)

# 9. EXPORTACIÓN AVANZADA CON METADATOS
# =====================================

def export_with_metadata(df_corrected, corrections, anomalies, output_dir):
    """Exporta el dataset con metadatos completos y reporte"""
    
    import os
    from datetime import datetime
    
    # Crear directorio si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    # 1. Exportar dataset limpio
    main_file = os.path.join(output_dir, 'dataset_evaluaciones_limpio.csv')
    df_corrected.to_csv(main_file, index=False, encoding='utf-8-sig')
    
    # 2. Crear reporte de metadatos
    metadata_file = os.path.join(output_dir, 'reporte_limpieza.txt')
    
    with open(metadata_file, 'w', encoding='utf-8') as f:
        f.write("="*80 + "\n")
        f.write("REPORTE DE LIMPIEZA Y CORRECCIÓN DE DATASET\n")
        f.write("="*80 + "\n")
        f.write(f"Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Archivo original: paste.txt\n")
        f.write(f"Archivo limpio: dataset_evaluaciones_limpio.csv\n\n")
        
        f.write("CORRECCIONES APLICADAS:\n")
        f.write("-" * 30 + "\n")
        if corrections:
            for i, correction in enumerate(corrections, 1):
                f.write(f"{i}. {correction}\n")
        else:
            f.write("No se requirieron correcciones.\n")
        
        f.write("\nANOMALÍAS DETECTADAS:\n")
        f.write("-" * 30 + "\n")
        if anomalies:
            for i, anomaly in enumerate(anomalies, 1):
                f.write(f"{i}. {anomaly}\n")
        else:
            f.write("No se detectaron anomalías significativas.\n")
        
        # Estadísticas finales
        f.write("\nESTADÍSTICAS FINALES:\n")
        f.write("-" * 30 + "\n")
        f.write(f"Total de registros: {len(df_corrected)}\n")
        f.write(f"Total de profesores únicos: {df_corrected['nombre_normalizado'].nunique()}\n")
        f.write(f"Total de departamentos: {df_corrected['departamento'].nunique()}\n")
        
        if 'calificacion_numerica' in df_corrected.columns:
            calificaciones = df_corrected['calificacion_numerica'].dropna()
            f.write(f"Calificación promedio: {calificaciones.mean():.2f}\n")
            f.write(f"Calificación mediana: {calificaciones.median():.2f}\n")
            f.write(f"Rango de calificaciones: {calificaciones.min():.2f} - {calificaciones.max():.2f}\n")
        
        # Completitud por columna
        f.write("\nCOMPLETITUD POR COLUMNA:\n")
        f.write("-" * 30 + "\n")
        completitud = (df_corrected.count() / len(df_corrected) * 100).sort_values(ascending=False)
        for col, pct in completitud.items():
            f.write(f"{col}: {pct:.1f}%\n")
    
    # 3. Exportar muestra de datos para validación
    sample_file = os.path.join(output_dir, 'muestra_validacion.csv')
    sample_df = df_corrected.sample(min(100, len(df_corrected)), random_state=42)
    sample_df.to_csv(sample_file, index=False, encoding='utf-8-sig')
    
    # 4. Crear diccionario de datos
    dict_file = os.path.join(output_dir, 'diccionario_datos.txt')
    
    with open(dict_file, 'w', encoding='utf-8') as f:
        f.write("DICCIONARIO DE DATOS - Dataset de Evaluaciones de Profesores\n")
        f.write("=" * 60 + "\n\n")
        
        column_descriptions = {
            'id': 'Identificador único del registro',
            'nombre_original': 'Nombre del profesor como apareció originalmente',
            'nombre_normalizado': 'Nombre del profesor estandarizado y limpio',
            'departamento': 'Departamento o división académica del profesor',
            'materia': 'Materia o curso impartido (si disponible)',
            'calificacion_numerica': 'Calificación numérica del profesor (escala 0-10)',
            'comentarios': 'Comentarios categorizados sobre el profesor',
            'resena_detallada': 'Reseña detallada (si disponible)',
            'archivo_fuente': 'Archivo de origen de los datos',
            'pagina': 'Página en el archivo fuente (si aplicable)',
            'num_fragmentos': 'Número de fragmentos de información',
            'fuente': 'Fuente de los datos (dataset1, dataset2, etc.)',
            'match_id': 'ID de coincidencia para registros relacionados',
            'match_score': 'Score de coincidencia (si aplicable)'
        }
        
        for col in df_corrected.columns:
            desc = column_descriptions.get(col, 'Descripción no disponible')
            dtype = str(df_corrected[col].dtype)
            unique_vals = df_corrected[col].nunique()
            missing_pct = (df_corrected[col].isnull().sum() / len(df_corrected)) * 100
            
            f.write(f"COLUMNA: {col}\n")
            f.write(f"  Descripción: {desc}\n")
            f.write(f"  Tipo de dato: {dtype}\n")
            f.write(f"  Valores únicos: {unique_vals}\n")
            f.write(f"  Valores faltantes: {missing_pct:.1f}%\n")
            
            # Mostrar algunos valores de ejemplo para columnas categóricas
            if df_corrected[col].dtype == 'object' and unique_vals <= 20 and unique_vals > 0:
                examples = df_corrected[col].dropna().unique()[:5]
                f.write(f"  Ejemplos: {', '.join(map(str, examples))}\n")
            
            f.write("\n")
    
    print(f"\n ARCHIVOS EXPORTADOS EN: {output_dir}")
    print(f"   dataset_evaluaciones_limpio.csv - Dataset principal limpio")
    print(f"   reporte_limpieza.txt - Reporte detallado de correcciones")
    print(f"   muestra_validacion.csv - Muestra de 100 registros para validar")
    print(f"   diccionario_datos.txt - Descripción de todas las columnas")
    
    return {
        'main_file': main_file,
        'metadata_file': metadata_file,
        'sample_file': sample_file,
        'dictionary_file': dict_file
    }

def create_executive_summary():
    """Crea un resumen ejecutivo del análisis"""
    
    print("\n" + "="*80)
    print("RESUMEN EJECUTIVO")
    print("="*80)
    
    print("""
 OBJETIVO COMPLETADO: Análisis integral y corrección de dataset de evaluaciones de profesores

 RESULTADOS CLAVE:
  • Dataset completamente limpio y estandarizado
  • Correcciones automáticas aplicadas según mejores prácticas
  • Análisis de sentimientos implementado
  • Detección de anomalías y patrones completada
  • Visualizaciones profesionales generadas

 MEJORAS IMPLEMENTADAS:
  • Normalización de nombres de departamentos
  • Validación de calificaciones numéricas
  • Limpieza de comentarios y formato
  • Eliminación de duplicados
  • Detección de inconsistencias

 VALOR AGREGADO:
  • Dashboard visual de calidad de datos
  • Análisis de correlaciones y patrones
  • Metadatos completos y documentación
  • Archivos listos para análisis posterior
  • Recomendaciones para mejoras futuras

 ENTREGABLES:
  • Dataset limpio en formato CSV
  • Reporte de correcciones aplicadas
  • Diccionario de datos completo
  • Muestra para validación manual
  • Visualizaciones y análisis estadístico

 PRÓXIMOS PASOS RECOMENDADOS:
  1. Validar muestra de datos manualmente
  2. Implementar pipeline de limpieza automática
  3. Desarrollar modelo de análisis de sentimientos específico
  4. Crear dashboard interactivo para monitoreo continuo
    """)

# Ejecutar exportación y resumen final
if df is not None:
    output_directory = r"# === NOTE: Replace with local path ==="
    exported_files = export_with_metadata(df_corrected, corrections, anomalies, output_directory)
    create_executive_summary()

print("\n" + "="*80)
print(" ANÁLISIS COMPLETO FINALIZADO CON ÉXITO")
print("="*80)

# Paso 2. Limpieza final

In [ ]:
import pandas as pd
import numpy as np
import re
import os

# Ruta del archivo original y destino
archivo_original = r"# === NOTE: Replace with local path ==="
carpeta_destino = r"# === NOTE: Replace with local path ==="
archivo_final = os.path.join(carpeta_destino, "evaluaciones_profesores_final.csv")

# Crear carpeta destino si no existe
os.makedirs(carpeta_destino, exist_ok=True)

def limpiar_texto(texto):
    """Limpia y normaliza texto"""
    if pd.isna(texto) or texto == '':
        return ''
    
    texto = str(texto).strip()
    # Remover caracteres especiales al inicio
    texto = re.sub(r'^[●•]\s*', '', texto)
    # Limpiar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto)
    return texto

def extraer_profesor_de_comentarios(comentarios):
    """Extrae nombre del profesor de los comentarios si está disponible"""
    if pd.isna(comentarios) or comentarios == '':
        return ''
    
    # Buscar patrones como "● NOMBRE: comentario"
    patron = r'^[●•]\s*([^:]+):'
    match = re.search(patron, str(comentarios))
    if match:
        return limpiar_texto(match.group(1))
    return ''

def procesar_csv():
    """Procesa y limpia el CSV"""
    print("Leyendo archivo CSV...")
    
    # Leer el CSV
    try:
        df = pd.read_csv(archivo_original, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(archivo_original, encoding='latin-1')
    
    print(f"Filas originales: {len(df)}")
    print(f"Columnas encontradas: {list(df.columns)}")
    
    # Crear DataFrame final con las columnas requeridas
    df_final = pd.DataFrame()
    
    # PROFESOR: Usar nombre_normalizado, si está vacío usar nombre_original
    # Si ambos están vacíos, extraer de comentarios
    df_final['PROFESOR'] = df['nombre_normalizado'].fillna('')
    mask_vacio = df_final['PROFESOR'] == ''
    df_final.loc[mask_vacio, 'PROFESOR'] = df.loc[mask_vacio, 'nombre_original'].fillna('')
    
    # Si aún está vacío, extraer de comentarios
    mask_aun_vacio = df_final['PROFESOR'] == ''
    for idx in df_final[mask_aun_vacio].index:
        profesor_extraido = extraer_profesor_de_comentarios(df.loc[idx, 'comentarios'])
        if profesor_extraido:
            df_final.loc[idx, 'PROFESOR'] = profesor_extraido
    
    # Limpiar nombres de profesores
    df_final['PROFESOR'] = df_final['PROFESOR'].apply(limpiar_texto)
    
    # ID
    df_final['ID'] = df['id']
    
    # MATERIA
    df_final['MATERIA'] = df['materia'].fillna('').apply(limpiar_texto)
    
    # DEPARTAMENTO
    df_final['DEPARTAMENTO'] = df['departamento'].fillna('').apply(limpiar_texto)
    
    # COMENTARIOS - Limpiar y combinar con calificación si existe
    df_final['COMENTARIOS'] = ''
    for idx in df.index:
        comentarios = limpiar_texto(df.loc[idx, 'comentarios'])
        calificacion = df.loc[idx, 'calificacion_numerica']
        
        # Si hay calificación, agregarla
        if not pd.isna(calificacion):
            if comentarios:
                comentarios = f"Calificación: {calificacion}/10 - {comentarios}"
            else:
                comentarios = f"Calificación: {calificacion}/10"
        
        df_final.loc[idx, 'COMENTARIOS'] = comentarios
    
    # RESEÑA_DETALLADA
    df_final['RESEÑA_DETALLADA'] = df['resena_detallada'].fillna('').apply(limpiar_texto)
    
    # Eliminar filas donde el profesor esté completamente vacío
    df_final = df_final[df_final['PROFESOR'] != ''].copy()
    
    # Eliminar duplicados basados en profesor y materia
    print("Eliminando duplicados...")
    df_final = df_final.drop_duplicates(subset=['PROFESOR', 'MATERIA'], keep='first')
    
    # Reorganizar por profesor
    df_final = df_final.sort_values(['PROFESOR', 'MATERIA']).reset_index(drop=True)
    
    print(f"Filas finales: {len(df_final)}")
    print(f"Profesores únicos: {df_final['PROFESOR'].nunique()}")
    print(f"Materias únicas: {df_final['MATERIA'].nunique()}")
    
    # Guardar archivo final
    df_final.to_csv(archivo_final, index=False, encoding='utf-8')
    print(f"\nArchivo guardado en: {archivo_final}")
    
    # Mostrar muestra de los datos
    print("\n=== MUESTRA DE LOS DATOS FINALES ===")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', 50)
    print(df_final.head(10))
    
    # Estadísticas finales
    print(f"\n=== ESTADÍSTICAS FINALES ===")
    print(f"Total de registros: {len(df_final)}")
    print(f"Profesores únicos: {df_final['PROFESOR'].nunique()}")
    print(f"Materias únicas: {df_final[df_final['MATERIA'] != '']['MATERIA'].nunique()}")
    print(f"Departamentos únicos: {df_final[df_final['DEPARTAMENTO'] != '']['DEPARTAMENTO'].nunique()}")
    print(f"Registros con comentarios: {(df_final['COMENTARIOS'] != '').sum()}")
    print(f"Registros con reseña detallada: {(df_final['RESEÑA_DETALLADA'] != '').sum()}")
    
    return df_final

def verificar_calidad_datos(df):
    """Verifica la calidad de los datos procesados"""
    print("\n=== VERIFICACIÓN DE CALIDAD ===")
    
    # Verificar profesores perdidos o mal formateados
    profesores_problematicos = df[df['PROFESOR'].str.len() < 3]['PROFESOR'].unique()
    if len(profesores_problematicos) > 0:
        print(f"  Profesores con nombres muy cortos: {profesores_problematicos}")
    
    # Verificar registros sin información útil
    sin_info = df[(df['COMENTARIOS'] == '') & (df['RESEÑA_DETALLADA'] == '')]
    print(f"Registros sin comentarios ni reseña: {len(sin_info)}")
    
    # Mostrar algunos ejemplos de profesores con más evaluaciones
    print("\n=== PROFESORES CON MÁS EVALUACIONES ===")
    conteos = df['PROFESOR'].value_counts().head(10)
    print(conteos)

# Ejecutar el procesamiento
if __name__ == "__main__":
    df_procesado = procesar_csv()
    verificar_calidad_datos(df_procesado)
    print(f"\n Proceso completado. Archivo final guardado en:")
    print(f"   {archivo_final}")

# Paso 3. Asociación del departamento

In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import os
import re

# Rutas de archivos
ruta_evaluaciones = r"# === NOTE: Replace with local path ==="
ruta_departamentos = r"# === NOTE: Replace with local path ==="
ruta_salida = r"# === NOTE: Replace with local path ==="

# Crear directorio de salida si no existe
os.makedirs(ruta_salida, exist_ok=True)

# Cargar los datasets
print("Cargando datos...")

# Cargar con parámetros más robustos para manejar CSVs problemáticos
try:
    df_evaluaciones = pd.read_csv(ruta_evaluaciones, 
                                encoding='utf-8',
                                quotechar='"',
                                quoting=1,  # QUOTE_ALL
                                skipinitialspace=True,
                                on_bad_lines='skip')
except:
    try:
        # Segundo intento con encoding diferente
        df_evaluaciones = pd.read_csv(ruta_evaluaciones, 
                                    encoding='latin-1',
                                    quotechar='"',
                                    quoting=1,
                                    skipinitialspace=True,
                                    on_bad_lines='skip')
    except:
        # Tercer intento: leer línea por línea y manejar errores
        print("Problemas con el CSV principal. Intentando lectura más robusta...")
        df_evaluaciones = pd.read_csv(ruta_evaluaciones, 
                                    encoding='utf-8',
                                    sep=',',
                                    quotechar='"',
                                    doublequote=True,
                                    skipinitialspace=True,
                                    on_bad_lines='warn',
                                    engine='python')

try:
    df_departamentos = pd.read_csv(ruta_departamentos, 
                                 encoding='utf-8',
                                 sep='\t',  # Usar tab como separador
                                 skipinitialspace=True)
except:
    try:
        df_departamentos = pd.read_csv(ruta_departamentos, 
                                     encoding='latin-1',
                                     sep='\t',
                                     skipinitialspace=True)
    except:
        # Si aún falla, separar manualmente la columna fusionada
        df_temp = pd.read_csv(ruta_departamentos, encoding='utf-8')
        col_name = df_temp.columns[0]
        df_departamentos = df_temp[col_name].str.split('\t', expand=True)
        df_departamentos.columns = ['NOMBRE', 'DEPARTAMENTO']

print(f"Evaluaciones cargadas: {len(df_evaluaciones)} registros")
print(f"Departamentos cargados: {len(df_departamentos)} registros")

# Verificar si necesitamos separar las columnas del archivo de departamentos
if len(df_departamentos.columns) == 1 and '\t' in df_departamentos.columns[0]:
    print("Detectando formato con tab, separando columnas...")
    col_name = df_departamentos.columns[0]
    df_departamentos = df_departamentos[col_name].str.split('\t', expand=True)
    df_departamentos.columns = ['NOMBRE', 'DEPARTAMENTO']
    print("Columnas separadas correctamente")

# Inspeccionar estructura de los datos
print("\nColumnas en evaluaciones:", list(df_evaluaciones.columns))
print("Columnas en departamentos:", list(df_departamentos.columns))

# Mostrar primeras filas para entender la estructura
print("\nPrimeras filas de evaluaciones:")
print(df_evaluaciones.head(2))
print("\nPrimeras filas de departamentos:")
print(df_departamentos.head(2))

# Función para limpiar y normalizar nombres
def limpiar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    # Convertir a string y limpiar
    nombre = str(nombre).upper().strip()
    # Remover caracteres especiales y espacios extra
    nombre = re.sub(r'[^\w\s]', ' ', nombre)
    nombre = ' '.join(nombre.split())
    return nombre

# Función para crear variaciones de nombres (apellido-nombre, nombre-apellido)
def crear_variaciones_nombre(nombre):
    if not nombre:
        return []
    
    partes = nombre.split()
    if len(partes) < 2:
        return [nombre]
    
    variaciones = [nombre]
    
    # Si tiene más de 2 partes, crear diferentes combinaciones
    if len(partes) >= 3:
        # Asumiendo que los primeros son nombres y los últimos apellidos
        mitad = len(partes) // 2
        nombre_parte = ' '.join(partes[:mitad])
        apellido_parte = ' '.join(partes[mitad:])
        
        # Agregar variaciones
        variaciones.extend([
            f"{apellido_parte} {nombre_parte}",
            f"{nombre_parte} {apellido_parte}"
        ])
        
        # También probar con solo los dos primeros y dos últimos
        if len(partes) > 2:
            variaciones.extend([
                f"{partes[-1]} {partes[0]}",  # último apellido + primer nombre
                f"{partes[0]} {partes[-1]}",  # primer nombre + último apellido
            ])
    
    return list(set(variaciones))  # Remover duplicados

# Limpiar nombres en ambos datasets
print("Limpiando nombres...")
df_evaluaciones['PROFESOR_LIMPIO'] = df_evaluaciones['PROFESOR'].apply(limpiar_nombre)
df_departamentos['NOMBRE_LIMPIO'] = df_departamentos['NOMBRE'].apply(limpiar_nombre)

# Crear diccionario de mapeo departamento -> división
mapeo_divisiones = {
    'DEPTO. DE FARMACOBIOLOGIA': 'División de Ciencias Básicas',
    'DEPTO. DE FISICA': 'División de Ciencias Básicas',
    'DEPTO. DE MATEMATICAS': 'División de Ciencias Básicas',
    'DEPTO. DE QUIMICA': 'División de Ciencias Básicas',
    
    'DEPTO. DE INGENIERIA CIVIL Y TOPOGRAFIA': 'División de Ingenierías',
    'DEPTO. DE INGENIERIA INDUSTRIAL': 'División de Ingenierías',
    'DEPTO. DE INGENIERIA MECANICA ELECTRICA': 'División de Ingenierías',
    'DEPTO. DE INGENIERIA DE PROYECTOS': 'División de Ingenierías',
    'DEPTO. DE INGENIERIA QUIMICA': 'División de Ingenierías',
    'DEPTO. DE MADERA, CELULOSA Y PAPEL': 'División de Ingenierías',
    
    'DEPTO DE BIOINGENIERIA TRASLACIONAL': 'División de Tecnologías para la Integración Ciber-Humana',
    'DEPTO. DE CIENCIAS COMPUTACIONALES': 'División de Tecnologías para la Integración Ciber-Humana',
    'DEPTO. DE INGENIERIA ELECTRO-FOTONICA': 'División de Tecnologías para la Integración Ciber-Humana',
    'DEPTO. DE INNOVACION BASADA EN LA INFORMACION Y EL CONOCIMIENTO': 'División de Tecnologías para la Integración Ciber-Humana'
}

# Función para encontrar la mejor coincidencia de nombre
def encontrar_mejor_coincidencia(nombre_profesor, lista_nombres_dept, umbral=80):
    if not nombre_profesor:
        return None, 0
    
    # Crear variaciones del nombre del profesor
    variaciones = crear_variaciones_nombre(nombre_profesor)
    
    mejor_coincidencia = None
    mejor_score = 0
    
    # Probar cada variación contra todos los nombres del departamento
    for variacion in variaciones:
        for nombre_dept in lista_nombres_dept:
            # Usar diferentes métodos de comparación
            scores = [
                fuzz.ratio(variacion, nombre_dept),
                fuzz.partial_ratio(variacion, nombre_dept),
                fuzz.token_sort_ratio(variacion, nombre_dept),
                fuzz.token_set_ratio(variacion, nombre_dept)
            ]
            
            score_max = max(scores)
            
            if score_max > mejor_score and score_max >= umbral:
                mejor_score = score_max
                mejor_coincidencia = nombre_dept
    
    return mejor_coincidencia, mejor_score

# Realizar la asociación
print("Realizando asociación de nombres...")
lista_nombres_dept = df_departamentos['NOMBRE_LIMPIO'].tolist()

resultados = []
no_encontrados = []

for idx, row in df_evaluaciones.iterrows():
    if idx % 100 == 0:
        print(f"Procesando registro {idx+1}/{len(df_evaluaciones)}")
    
    nombre_profesor = row['PROFESOR_LIMPIO']
    mejor_match, score = encontrar_mejor_coincidencia(nombre_profesor, lista_nombres_dept)
    
    if mejor_match:
        # Encontrar el departamento correspondiente
        dept_info = df_departamentos[df_departamentos['NOMBRE_LIMPIO'] == mejor_match].iloc[0]
        departamento = dept_info['DEPARTAMENTO']
        division = mapeo_divisiones.get(departamento, 'División no encontrada')
        
        resultados.append({
            'match_encontrado': True,
            'departamento': departamento,
            'division': division,
            'score': score
        })
    else:
        no_encontrados.append(nombre_profesor)
        resultados.append({
            'match_encontrado': False,
            'departamento': 'No encontrado',
            'division': 'No encontrado',
            'score': 0
        })

# Agregar resultados al dataframe principal
df_resultado = df_evaluaciones.copy()
for i, resultado in enumerate(resultados):
    df_resultado.loc[i, 'DEPARTAMENTO'] = resultado['departamento']
    df_resultado.loc[i, 'DIVISION'] = resultado['division']
    df_resultado.loc[i, 'SCORE_MATCH'] = resultado['score']

# Consolidar columnas de reseñas (COMENTARIOS y RESEÑA_DETALLADA)
print("Consolidando reseñas...")
def consolidar_resenas(row):
    comentarios = str(row['COMENTARIOS']) if pd.notna(row['COMENTARIOS']) else ""
    resena = str(row['RESEÑA_DETALLADA']) if pd.notna(row['RESEÑA_DETALLADA']) else ""
    
    # Combinar ambas, eliminando duplicados si son iguales
    if comentarios == resena:
        return comentarios
    elif comentarios and resena:
        return f"{comentarios} {resena}"
    else:
        return comentarios or resena

df_resultado['RESENA_COMPLETA'] = df_resultado.apply(consolidar_resenas, axis=1)

# Limpiar dataset final
print("Limpiando dataset final...")
columnas_finales = ['PROFESOR', 'MATERIA', 'DEPARTAMENTO', 'DIVISION', 'RESENA_COMPLETA']
df_final = df_resultado[columnas_finales].copy()

# Renombrar columnas para mayor claridad
df_final.columns = ['PROFESOR', 'MATERIA', 'DEPARTAMENTO', 'DIVISION', 'COMENTARIOS']

# Guardar resultado
archivo_salida = os.path.join(ruta_salida, "evaluaciones_con_departamentos.csv")
df_final.to_csv(archivo_salida, index=False, encoding='utf-8')

# Guardar también estadísticas del proceso
estadisticas = {
    'total_registros': len(df_evaluaciones),
    'matches_encontrados': sum(1 for r in resultados if r['match_encontrado']),
    'no_encontrados': len(no_encontrados),
    'porcentaje_exito': (sum(1 for r in resultados if r['match_encontrado']) / len(df_evaluaciones)) * 100
}

# Mostrar estadísticas
print("\n" + "="*50)
print("RESUMEN DEL PROCESO")
print("="*50)
print(f"Total de registros procesados: {estadisticas['total_registros']}")
print(f"Matches encontrados: {estadisticas['matches_encontrados']}")
print(f"No encontrados: {estadisticas['no_encontrados']}")
print(f"Porcentaje de éxito: {estadisticas['porcentaje_exito']:.2f}%")

# Mostrar distribución por división
print("\nDISTRIBUCIÓN POR DIVISIÓN:")
print("-" * 30)
division_counts = df_final['DIVISION'].value_counts()
for division, count in division_counts.items():
    print(f"{division}: {count}")

# Guardar lista de no encontrados para revisión
if no_encontrados:
    with open(os.path.join(ruta_salida, "profesores_no_encontrados.txt"), 'w', encoding='utf-8') as f:
        f.write("PROFESORES NO ENCONTRADOS:\n")
        f.write("="*40 + "\n")
        for nombre in set(no_encontrados):  # Usar set para evitar duplicados
            f.write(f"{nombre}\n")

print(f"\nArchivo principal guardado en: {archivo_salida}")
print(f"Directorio de salida: {ruta_salida}")
print("¡Proceso completado exitosamente!")